In [8]:
import os
import json

### 각 클래스별 정상 이상 태스크 명들 읽어오기

In [34]:
os.chdir('/home/daein/Jupyter_notebook/data_tmp')
class_li = os.listdir()

In [35]:
# 각 클래스별 태스크 명 가져오는 함수
def get_taskName(state):
    class_dict_tmp = {}
    
    for i in class_li:
        try:
            class_dict_tmp[i] = os.listdir(os.path.join('./', i, state))
            print(i + ' 클래스 {} 태스크 리스트 추출 완료\n'.format(state), '-' * 50)

        except:
            print(i + ' 클래스 일부에 {} 태스크가 없음'.format(state))
            
    return class_dict_tmp

# class_dict_normal = {}
# class_dict_normal = get_taskName('정상')

# class_dict_abnormal = {}
# class_dict_abnormal = get_taskName('이상')

class_dict = {}
class_dict = get_taskName('')

### 각 클래스 태스크 별 이미지 개수, 존재 여부 등을 검사

In [19]:
# class_dict_normal : 정상 클래스의 태스크 리스트
# class_dict_abnormal : 비정상 클래스의 태스크 리스트

In [20]:
# json 파일에서 이미지 명, 클래스 명 빼오는 함수
def load_json_imageList(file):
    json_in_imageName = []
    json_in_classId = []
    json_in_className = []
    
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        images_li = data['images']
        cate_li = data['categories']
        labelMap_li = data['categories']
    
        for i in range(len(images_li)):
            json_in_imageName.append(images_li[i]['file_name'])
            
        for i in range(len(cate_li)):
            json_in_className.append(cate_li[i]['name'])
            json_in_classId.append(cate_li[i]['id'])
            
    return json_in_imageName, json_in_className, json_in_classId, labelMap_li

In [33]:
class_labelMap = {}

# Json 파일 내의 이미지 파일 이름을 가져오는 함수
def inner_ImglistToJson(status, class_dict):
    f = open('./data_check.txt', 'w')
    
    for i in class_dict:
        for j in class_dict[i]:
            try:
                # json 내 이미지 리스트 추출
                jsonName = os.listdir(os.path.join(i, status, j, 'annotations'))
                json_result = load_json_imageList(os.path.join(i, status, j, 'annotations', jsonName[0]))
                json_in_imageName = json_result[0]
                json_in_className = json_result[1]
                json_in_labelMap = json_result[3]
                
                if i in class_labelMap.keys(): 
                    class_labelMap[i].append([json_result[2][0], json_result[1][0]])
                else:
                    class_labelMap[i] = []
                    class_labelMap[i].append([json_result[2][0], json_result[1][0]])
                    
                real_imageName = os.listdir(os.path.join(i, status, j, 'images'))
                
#                 print('검사 클래스 명 : \t', i), 
#                 print('검사 태스크 명 : \t', j)
#                 print('json 파일 내의 이미지 개수 : \t', len(json_in_imageName))
#                 print('json 파일 내의 클래스 명 : \t', json_in_className)
#                 print('json 파일 내의 라벨맵 : \t', json_in_labelMap)
#                 print('실제 이미지 개수 : \t', len(real_imageName))
#                 print('실제 파일명과 동일 여부 : \t', '같음' if (sorted(json_in_imageName) == sorted(real_imageName)) else '다름')
#                 print('-'*50)

                f.write('검사 클래스 명 : \t' + i + '\n'), 
                f.write('검사 태스크 명 : \t' + j + '\n')
                f.write('json 파일 내의 이미지 개수 : \t' + str(len(json_in_imageName)) + '\n')
                f.write('json 파일 내의 클래스 명 : \t' + str(json_in_className) + '\n')
                f.write('json 파일 내의 라벨맵 : \t' + str(json_in_labelMap) + '\n')
                f.write('실제 이미지 개수 : \t' + str(len(real_imageName)) + '\n')
                f.write('실제 파일명과 동일 여부 : \t' + ('같음' if (sorted(json_in_imageName) == sorted(real_imageName)) else '다름'))
                f.write('\n')
                f.write('-'*50)
                f.write('\n')
                
                
            except:
                print(i + ' 클래스 일부에 {} 태스크가 없음\n'.format(status), '-' * 50)
                
    f.close()
                
inner_ImglistToJson('', class_dict)

### 라벨맵 중복 확인 (미완성)

In [36]:
for i in class_li:
    class_labelMap[i] = list(set([tuple(set(i)) for i in class_labelMap[i]]))
    print(i + ' :',class_labelMap[i])